In [1]:
# ==============================================================================
# CELL 1: VALIDATION DATA PREPARATION
# ==============================================================================
import os
import shutil
import torch
import numpy as np
from PIL import Image
from tqdm import tqdm
from scipy.ndimage import label as cc_label
from google.colab import drive

# 1. MOUNT DRIVE
if not os.path.exists('/content/drive'):
    print("🔌 Mounting Google Drive...")
    drive.mount('/content/drive', force_remount=True)

# 2. CONFIGURATION
DRIVE_ROOT = "/content/drive/MyDrive/Latitude_AI_Team/data"
VAL_MASK_DIR = f"{DRIVE_ROOT}/bdd100k_segmentations/bdd100k_seg_maps_2/labels/val"
VAL_IMG_DIR = f"{DRIVE_ROOT}/bdd100k_images_10k/10k/val"
ZIP_PATH = os.path.join(DRIVE_ROOT, "bdd100k_fast_cache.zip")

# Local Dirs
DATA_ROOT = "/content/data"
SAVE_DIR_PT = os.path.join(DATA_ROOT, "precomputed_masks")
SAVE_DIR_IMG = os.path.join(DATA_ROOT, "images/val")

os.makedirs(SAVE_DIR_PT, exist_ok=True)
os.makedirs(SAVE_DIR_IMG, exist_ok=True)

# 3. RESTORE FROM BACKUP (If available)
if os.path.exists(ZIP_PATH):
    print(f" Found backup zip. Unzipping to speed up process...")
    try:
        shutil.unpack_archive(ZIP_PATH, DATA_ROOT)
        print(" Unzipped backup.")
    except:
        print(" Zip failed (might be corrupt). Proceeding with generation.")

# 4. FILL MISSING VALIDATION DATA (Ensure we have all ~1000)
print(f" Scanning {VAL_MASK_DIR} for full validation set...")
all_files = sorted([f for f in os.listdir(VAL_MASK_DIR) if f.endswith('.png')])
print(f" Found {len(all_files)} validation images.")

BDD_IDS = [6, 7, 11, 13, 12, 14, 15, 17, 18]
ID_TO_LABEL = {bdd_id: i+1 for i, bdd_id in enumerate(BDD_IDS)}

print("⚙️ Processing any missing files...")
for mask_file in tqdm(all_files):
    try:
        basename = mask_file.replace("_train_id.png", "")
        pt_path = os.path.join(SAVE_DIR_PT, basename + ".pt")

        # Check if we already have this file (from zip or previous run)
        # AND check if the image exists locally
        dst_img = os.path.join(SAVE_DIR_IMG, basename + ".jpg")
        if os.path.exists(pt_path) and os.path.exists(dst_img):
            continue

        # Generate .pt if missing
        mask_path = os.path.join(VAL_MASK_DIR, mask_file)
        mask_np = np.array(Image.open(mask_path).convert("L"), dtype=np.int32)

        boxes, labels, final_masks = [], [], []
        for bdd_id in BDD_IDS:
            class_mask = (mask_np == bdd_id).astype(np.uint8)
            if class_mask.sum() < 10: continue

            inst_map, ninst = cc_label(class_mask)
            for i in range(1, ninst+1):
                pos = np.where(inst_map == i)
                ymin, xmin = np.min(pos[0]), np.min(pos[1])
                ymax, xmax = np.max(pos[0]), np.max(pos[1])
                if (ymax-ymin)<5 or (xmax-xmin)<5: continue

                boxes.append([xmin, ymin, xmax, ymax])
                labels.append(ID_TO_LABEL[bdd_id])
                final_masks.append((inst_map == i).astype(np.uint8))

        if boxes:
            torch.save({
                "boxes": torch.tensor(boxes, dtype=torch.float32),
                "labels": torch.tensor(labels, dtype=torch.int64),
                "masks": torch.tensor(np.array(final_masks), dtype=torch.uint8)
            }, pt_path)

            # Copy Image
            src_img = os.path.join(VAL_IMG_DIR, basename + ".jpg")
            if os.path.exists(src_img): shutil.copy(src_img, dst_img)

    except Exception as e:
        pass

print("\n✅ FULL VALIDATION SET READY.")

🔌 Mounting Google Drive...
Mounted at /content/drive
📦 Found backup zip. Unzipping to speed up process...
✅ Unzipped backup.
🚀 Scanning /content/drive/MyDrive/Latitude_AI_Team/data/bdd100k_segmentations/bdd100k_seg_maps_2/labels/val for full validation set...
📋 Found 1000 validation images.
⚙️ Processing any missing files...


100%|██████████| 1000/1000 [00:14<00:00, 68.04it/s]


✅ FULL VALIDATION SET READY.


In [2]:
# ==============================================================================
# CELL 2: FINAL EVALUATION ON 1000 IMAGES
# ==============================================================================
!pip install torchmetrics -q

import torch
import os
import torchvision
import torchvision.transforms as T
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
from torchmetrics.detection import MeanAveragePrecision
from tqdm import tqdm

# 1. SETUP
DEVICE = torch.device("cuda") if torch.cuda.is_available() else "cpu"
print(f" Evaluating on: {DEVICE}")

# Paths
PT_DIR = "/content/data/precomputed_masks"
VAL_IMG_DIR = "/content/data/images/val"

# Checkpoint Search (Drive or Local)
CHECKPOINT_FILENAME = "best_model_checkpoint_full_run.pth"
DRIVE_MODELS_DIR = "/content/drive/MyDrive/Latitude_AI_Team/models"
DRIVE_PATH = os.path.join(DRIVE_MODELS_DIR, CHECKPOINT_FILENAME)
# Fallback if it was saved in root
LEGACY_PATH = "/content/drive/MyDrive/Latitude_AI_Team/best_model_checkpoint_full_run.pth"

MODEL_PATH = None
if os.path.exists(DRIVE_PATH): MODEL_PATH = DRIVE_PATH
elif os.path.exists(LEGACY_PATH): MODEL_PATH = LEGACY_PATH

if not MODEL_PATH:
    raise FileNotFoundError("❌ Could not find checkpoint file in Drive!")

# 2. DATASET CLASS
class BDDPrecomputedDataset(Dataset):
    def __init__(self, img_dir, pt_dir, basenames):
        self.img_dir = img_dir; self.pt_dir = pt_dir; self.basenames = basenames
    def __len__(self): return len(self.basenames)
    def __getitem__(self, idx):
        basename = self.basenames[idx]
        img = Image.open(os.path.join(self.img_dir, basename + ".jpg")).convert("RGB")
        data = torch.load(os.path.join(self.pt_dir, basename + ".pt"))
        target = {"boxes": data["boxes"], "labels": data["labels"], "masks": data["masks"], "image_id": torch.tensor([idx])}
        return T.ToTensor()(img), target

def collate_fn(batch): return tuple(zip(*batch))

# 3. LOAD DATA
# Get all validation files that exist locally
val_files = sorted([f.replace('.jpg', '') for f in os.listdir(VAL_IMG_DIR)
                    if os.path.exists(os.path.join(PT_DIR, f.replace('.jpg', '.pt')))])

print(f"Evaluating on {len(val_files)} images.")

val_loader = DataLoader(
    BDDPrecomputedDataset(VAL_IMG_DIR, PT_DIR, val_files),
    batch_size=1, shuffle=False, num_workers=2, collate_fn=collate_fn, pin_memory=True
)

# 4. LOAD MODEL
BDD_IDS = [6, 7, 11, 13, 12, 14, 15, 17, 18]
model = torchvision.models.detection.maskrcnn_resnet50_fpn(weights=None) # No pre-train needed, we load ours
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, 1 + len(BDD_IDS))
in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask, 256, 1 + len(BDD_IDS))

print(f" Loading weights from: {MODEL_PATH}")
ckpt = torch.load(MODEL_PATH, map_location=DEVICE)
model.load_state_dict(ckpt['model_state_dict'])
model.to(DEVICE)
model.eval()

# 5. RUN METRICS
print(" Starting Inference Loop...")
metric = MeanAveragePrecision(box_format="xyxy", iou_type="segm").to(DEVICE)

with torch.no_grad():
    for i, (images, targets) in tqdm(enumerate(val_loader), total=len(val_loader)):
        images = [img.to(DEVICE) for img in images]
        targets = [{k: v.to(DEVICE) for k, v in t.items()} for t in targets]

        outputs = model(images)
        processed = []
        for pred in outputs:
            if 'masks' in pred: pred['masks'] = (pred['masks'] > 0.5).squeeze(1).to(torch.uint8)
            processed.append(pred)
        metric.update(processed, targets)

print("\nComputing Final Scores...")
res = metric.compute()

print("\n" + "="*50)
print(" OFFICIAL VALIDATION RESULTS")
print("="*50)
print(f"AP @[.50:.95]: {res['map'].item():.4f}")
print(f"AP @ .50:      {res['map_50'].item():.4f}")
print(f"AP @ .75:      {res['map_75'].item():.4f}")
print(f"AP (Small):    {res['map_small'].item():.4f}")
print("="*50)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 16.4 MB/s eta 0:00:00
🚀 Evaluating on: cuda
📊 Evaluating on 988 images.
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


100%|██████████| 97.8M/97.8M [00:00<00:00, 227MB/s]


🔄 Loading weights from: /content/drive/MyDrive/Latitude_AI_Team/models/best_model_checkpoint_full_run.pth
🚀 Starting Inference Loop...


100%|██████████| 988/988 [03:51<00:00,  4.28it/s]



Computing Final Scores...

🏆 OFFICIAL VALIDATION RESULTS
AP @[.50:.95]: 0.2702
AP @ .50:      0.4705
AP @ .75:      0.2653
AP (Small):    0.1305
